# The Guide using easy Job Queue with Python and Flask

In [ ]:
# Intsall Redis
$ wget http://download.redis.io/releases/redis-5.0.0.tar.gz
$ tar xzf redis-5.0.0.tar.gz
$ cd redis-5.0.0
$ make

# Run redis server
$ src/redis-server

In [ ]:
# Access your project, open console and run:
# source git of rq: https://github.com/rq/rq
$ pip install rq flask flask_restful
$ pip freeze > requirements.txt

In [ ]:
# Set up the Worker
# Let’s start by creating a worker process to listen for queued tasks. 
# Create a new file worker.py, and add this code:
import os

import redis
from rq import Worker, Queue, Connection

listen = ['default']

redis_url = os.getenv('REDISTOGO_URL', 'redis://localhost:6379')

conn = redis.from_url(redis_url)

if __name__ == '__main__':
    with Connection(conn):
        worker = Worker(list(map(Queue, listen)))
        worker.work()

# Here, we listened for a queue called default and established a connection 
# to the Redis server on localhost:6379.

In [ ]:
# open another console and run:
$ python worker.py

# if you run with production or server online, you can make a file worker.sh and add code:

#!/bin/bash
$ source activate your_env
$ python worker.py

# Now we need to add our app.py to send jobs to the queue…

In [ ]:
# Add some code for app.py file:

from flask import Flask, request, jsonify
from flask_restful import Resource, Api
from rq import Queue
from worker import conn

app = Flask(__name__)
api = Api(app)

# set up a Redis connection and initialized a queue based on that connection.
q = Queue(connection=conn)

job = q.enqueue(your_function, param_1, param_2) # you can add more params if you want

# The blocking function will automatically be executed in the background worker process.

In [ ]:
# Monitoring
# View with brower, you can install rq-dashboard:
$ pip install rq-dashboard
$ rq-dashboard

# show info worker with the console
$ rq info
# or show detail:
$ rq info -R
# You can specify a poll interval, by using the --interval flag.
$ rq info --interval 1

# Manage jobs with console
# Display all pending job IDs in the default queue. 
# We’re using the Redis DB number 2, the default RQ queue on a standard Pootle install.
$ redis-cli -n 2 lrange rq:queue:default 0 -1
03135097-00f8-46eb-b084-6f34a16d9940
a07309b3-f056-47e7-856c-c608bda2f171
3df6a559-2e3c-4c0c-b09c-1948b4bacda2

# Display the failed job IDs.
$ redis-cli -n 2 lrange rq:queue:failed 0 -1

# To investigate a failed job simply add rq:job: prefix to a job ID and use a command such as this
# This will allow you to see any traceback and investigate and solve them.
$ redis-cli -n 2 hgetall rq:job:60ed13df-0ce5-4b98-96f0-f8e0294ba421

# delete fail job:
$ redis-cli -n 2 DEL "rq:queue:failed"